![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Agentic tasks with AutoGen and Redis

This notebook demonstrates how to build an agent using Microsoft's [AutoGen](https://microsoft.github.io/autogen/stable//index.html) agent framework and the RedisMemory integration.

We'll define an agent, give it access to tools and memory, then set in on a task to see how it uses its abilities.

## Let's Begin!

<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/04_autogen_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Install packages and set up Redis

In [1]:
%pip install -q autogen autogen_agentchat
%pip install -q "autogen-ext[redisvl]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### For Colab download and run a Redis instance

In [2]:
# NBVAL_SKIP
'''
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes
'''

'\n%%sh\ncurl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg\necho "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list\nsudo apt-get update  > /dev/null 2>&1\nsudo apt-get install redis-stack-server  > /dev/null 2>&1\nredis-stack-server --daemonize yes\n'

In [3]:
import os
from typing import List
import requests
import json

In [4]:
# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


## Building our agent

We'll be building a product review writing agent that takes in a set of product descriptions, collects relevant information, and provides a summary and analysis you can use for SEO.


### Defining tools
One of the defining the features of an agent is its ability to use tools so let's give it some.

For use with Autogen that just requires we define a well named function with type hints in its signature

In [5]:

async def summarize(all_reviews: List[str]) -> str:
    """takes in a list of restaurant reviews and returns a single summary of all of them."""
    # set up open ai client
    
    # concatenate all the reviews together
   
    # pass all the reviews along with a system prompt asking for a summary
    return ' summary '


async def get_keywords(full_text: str) -> List[str]:
    """extract the most commonly occurring keywords present in the reviews to know
    which terms it is likely to rank highly for in keyword search engines."""
    import re
    from collections import Counter
    # define a set of common English stopwords to ignore
    STOPWORDS = {
        'the', 'of', 'and', 'to', 'for', 'in', 'on', 'at', 'a', 'an', 'is', 'it', 'with', 'as', 'by', 'from', 'that',
        'this', 'be', 'are', 'was', 'were', 'or', 'but', 'not', 'so', 'if', 'then', 'than', 'which', 'who', 'whom',
        'about', 'into', 'out', 'up', 'down', 'over', 'under', 'again', 'further', 'once', 'here', 'there', 'when',
        'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
        'nor', 'only', 'own', 'same', 'can', 'will', 'just', 'don', 'should', 'now', 'has', 'have', 'had', 'do', 'does',
        'did', 'their', 'them', 'they', 'you', 'your', 'yours', 'he', 'him', 'his', 'she', 'her', 'hers', 'we', 'us',
        'our', 'ours', 'i', 'me', 'my', 'mine', 'also'
    }
    # remove punctuation and lowercase the text
    words = re.findall(r'\b\w+\b', full_text.lower())
    # filter out stopwords
    filtered_words = [word for word in words if word not in STOPWORDS]
    # count occurrences
    word_counts = Counter(filtered_words)
    # return the top 10
    return [word for word, _ in word_counts.most_common(10)]


## Adding relevant memories
Our agent needs to know what people think of these restaurants so we'll add the user reviews to our agent memory powered by Redis.


In [6]:
# fetch the reviews from our public S3 bucket
def fetch_data(file_name):
    dataset_path = 'datasets/'
    try:
        with open(dataset_path + file_name, 'r') as f:
            return json.load(f)
    except:
        url = 'https://redis-ai-resources.s3.us-east-2.amazonaws.com/recommenders/datasets/two-towers/'
        r = requests.get(url + file_name)
        if not os.path.exists(dataset_path):
            os.makedirs(dataset_path)
        with open(dataset_path + file_name, 'wb') as f:
            f.write(r.content)
        return json.loads(r.content.decode('utf-8'))

In [7]:
# the original dataset can be found here: https://www.kaggle.com/datasets/jkgatt/restaurant-data-with-100-trip-advisor-reviews-each

restaurant_data = fetch_data('factual_tripadvisor_restaurant_data_all_100_reviews.json')

print(f"we have {restaurant_data['restaurant_count']} restaurants in our dataset, with {restaurant_data['total_review_count']} total reviews")

restaurant_reviews = restaurant_data["restaurants"] # ignore the count fields

# drop some of the fields that don't need
for restaurant in restaurant_reviews:
    for field in ['region', 'country', 'tel', 'fax', 'email', 'website', 'address_extended', 'chain_name', 'trip_advisor_url']:
        if field in restaurant:
            restaurant.pop(field)

we have 147 restaurants in our dataset, with 14700 total reviews


In [ ]:
from logging import WARNING, getLogger

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory.redis import RedisMemory, RedisMemoryConfig
from autogen_ext.models.openai import OpenAIChatCompletionClient

logger = getLogger()
logger.setLevel(WARNING)

# initailize Redis memory
redis_memory = RedisMemory(
    config=RedisMemoryConfig(
        redis_url="redis://localhost:6379",
        index_name="restaurant_reviews",
        prefix="trip_advisor",
    )
)

for restaurant in restaurant_reviews[:10]:
    # add each review to our agent memory
    for review in restaurant['reviews']:
        try:
            await redis_memory.add(
                MemoryContent(
                    content=review.pop("review_title") +" " + review.pop("review_text"),
                    mime_type=MemoryMimeType.TEXT,
                    metadata=review,
                )
            )
        except KeyError:
            print(review.keys())

/Users/justin.cechmanek/.pyenv/versions/3.11.9/envs/redis-ai-res/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create our agent and set it on a task

In [9]:

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Create assistant agent with ChromaDB memory
assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[summarize , get_keywords],
    memory=[redis_memory],
)

agent_task = "Write an article for me reviewing the restaurants in the San Francisco bay area. \
            Group them into categories based on their cuisine and price, and talk about the \
            top rated restaurants in each category."

stream = assistant_agent.run_stream(task=agent_task)
await Console(stream)


---------- TextMessage (user) ----------
Write an article for me reviewing the restaurants in the San Francisco bay area.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.
---------- MemoryQueryEvent (assistant_agent) ----------
[MemoryContent(content="Bar Tartine impressed my VIP guests, and left me yearning for a return trip to San Francisco We planned a private business party at Bar Tartine and 'bought' the restaurant for 3 hours. We invited VIP customers and prospective partners and lured them there with the promise of amazing, unique food. The food, the staff, the Chefs all exceeded our expectations and gave our guests one of the most memorable dining experiences we have ever had. We left the menu in the hands of the Chefs and it was exquisite. The evening was filled with debates over which dish was the best and there was little consensus because every single offering was unique, beautifu

TaskResult(messages=[TextMessage(id='2ed8b0d5-e54f-4825-980e-797e81df7a31', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 30, 23, 57, 46, 948689, tzinfo=datetime.timezone.utc), content='Write an article for me reviewing the restaurants in the San Francisco bay area.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.', type='TextMessage'), MemoryQueryEvent(id='33e6b31c-0c40-4ec8-9c60-26738eee82da', source='assistant_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 30, 23, 57, 46, 987869, tzinfo=datetime.timezone.utc), content=[MemoryContent(content="Bar Tartine impressed my VIP guests, and left me yearning for a return trip to San Francisco We planned a private business party at Bar Tartine and 'bought' the restaurant for 3 hours. We invited VIP customers and prospective partners and lured them there with the promise of amazing,

### Clean up
close out our agent and empty our Redis index

In [10]:

#await model_client.close()
#await redis_memory.close()